In [1]:
pip install Jetson.GPIO

  Using cached https://files.pythonhosted.org/packages/06/48/3461141a3af476772b1112ce3367888e9209d59cbdf48a95c696dd21276b/jetson_gpio-2.1.11-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [3]:
# --- Cell 5: Stop / Cleanup ---
camera.running = False
stop()
GPIO.cleanup()
print("Motors stopped and GPIO cleaned up.")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Prediction: sleep (0.85)
Alert: Driver sleeping! Motors stopped.
Motors stopped and GPIO cleaned up.


In [2]:
# --- Imports ---
import Jetson.GPIO as GPIO
import time
import ipywidgets as widgets
from IPython.display import display, clear_output
from jetcam.utils import bgr8_to_jpeg
from jetcam.usb_camera import USBCamera 
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2

# --- GPIO Setup ---
# Motor GPIO Pins (BOARD numbering)
IN1, IN2 = 29, 31  # Left motor
IN3, IN4 = 35, 37  # Right motor

GPIO.setmode(GPIO.BOARD)  # Must be called before any GPIO operations
GPIO.setup([IN1, IN2, IN3, IN4], GPIO.OUT)
GPIO.setwarnings(False)

# --- Motor Functions ---
def left_forward():
    GPIO.output(IN1, GPIO.HIGH)
    GPIO.output(IN2, GPIO.LOW)

def left_backward():
    GPIO.output(IN1, GPIO.LOW)
    GPIO.output(IN2, GPIO.HIGH)

def left_stop():
    GPIO.output(IN1, GPIO.LOW)
    GPIO.output(IN2, GPIO.LOW)

def right_forward():
    GPIO.output(IN3, GPIO.HIGH)
    GPIO.output(IN4, GPIO.LOW)

def right_backward():
    GPIO.output(IN3, GPIO.LOW)
    GPIO.output(IN4, GPIO.HIGH)

def right_stop():
    GPIO.output(IN3, GPIO.LOW)
    GPIO.output(IN4, GPIO.LOW)

def forward():
    left_forward()
    right_forward()

def backward():
    left_backward()
    right_backward()

def stop():
    left_stop()
    right_stop()

# --- ML Setup ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

TRANSFORMS = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

def preprocess(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    tensor = TRANSFORMS(frame)
    tensor = tensor.unsqueeze(0).to(device)
    return tensor

MODEL_PATH = "model/driver_sleep_resnet18.pth"
CATEGORIES = ['awake', 'sleep']
SLEEP_THRESHOLD = 2

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, len(CATEGORIES))
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.to(device)
model.eval()

# --- Camera Setup ---
camera = USBCamera(width=640, height=480, capture_device=0)
image_widget = widgets.Image(format='jpeg')
display(image_widget)

sleep_frame_count = 0

# --- Callback Function ---
def update_image(change):
    global sleep_frame_count
    frame = change['new']
    
    # Update live preview
    image_widget.value = bgr8_to_jpeg(frame)
    
    # ML inference
    input_tensor = preprocess(frame)
    with torch.no_grad():
        output = model(input_tensor)
        probs = F.softmax(output, dim=1)
        pred_idx = torch.argmax(probs, dim=1).item()
        pred_label = CATEGORIES[pred_idx]
        pred_prob = probs[0, pred_idx].item()
    
    # Track sleep frames
    if pred_label == 'sleep':
        sleep_frame_count += 1
    else:
        sleep_frame_count = 0
    
    # Motor control
    if sleep_frame_count >= SLEEP_THRESHOLD:
        stop()
        alert_msg = "Alert: Driver sleeping! Motors stopped."
    else:
        forward()
        alert_msg = "Driver awake - moving forward."
    
    # Display output
    clear_output(wait=True)
    display(image_widget)
    print(f"Prediction: {pred_label} ({pred_prob:.2f})")
    print(alert_msg)

# --- Start Camera Loop ---
camera.observe(update_image, names='value')
camera.running = True

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Prediction: sleep (0.87)
Alert: Driver sleeping! Motors stopped.
